In [329]:
import pandas as pd
import streamlit as st
import pipreqs
import numpy as np
import csv

In [330]:
league = 'SBC'
season = 2040
curr_date = '2040-05-28'
league_al_name = 'AL' 
league_nl_name = 'NL'

# NBC Tournament only. This needs to be updated once annually
tourney_start = '2023-07-04'
tourney_end = '2023-07-25'

path = 'C:/Users/night/OneDrive/Documents/Out of the Park Developments/OOTP Baseball 22/saved_games/'

In [26]:
leagues = {
    "SBC":[145, 
           '2040-03-29', 
           2040,
           path + 'SBC.lg/import_export/csv/'],
    "PBC":[146, 
           '2030-10-28',
           2030,
           path + 'PBC.lg/import_export/csv/'],
    "NBC":[153, 
           '2023-04-11',
           2024,
           path + 'NBC.lg/import_export/csv/'],
    "MVP":[153, 
           '2024-03-28', 
           2024,
           path + 'MVP.lg/import_export/csv/']
}

In [40]:
def comp_balance(league):
    team_history = pd.read_csv(leagues[league][3] + 'team_history.csv')
    team_history_financials = pd.read_csv(leagues[league][3] + 'team_history_financials.csv')
    team_history = team_history[(team_history['year'] == (leagues[league][2] - 1)) & 
                            (team_history['league_id'] == leagues[league][0])].set_index('team_id')

    team_history = team_history[['name', 'nickname', 'made_playoffs']]

    team_history_financials = team_history_financials[(team_history_financials['year'] == (leagues['SBC'][2] - 1)) & 
                            (team_history_financials['league_id'] == leagues['SBC'][0])].set_index('team_id')

    temp = pd.merge(team_history, team_history_financials, how = 'left', left_index = True, right_index = True)
    
    comp_bal = pd.DataFrame()
    comp_bal['Team'] = temp['name'] + ' ' + temp['nickname']
    comp_bal['Total Revenue'] = temp['gate_revenue'] + \
                                temp['season_ticket_revenue'] + \
                                temp['media_revenue'] + \
                                temp['merchandising_revenue'] + \
                                temp['playoff_revenue']
    comp_bal['Media Revenue'] = temp['media_revenue'] / 1000000
    comp_bal['Media Revenue %'] = round(((comp_bal['Media Revenue'] * 1000000)/comp_bal['Total Revenue'])*100,1)
    comp_bal['Other Revenue %'] = 100 - comp_bal['Media Revenue %']
    comp_bal['Market Size'] = temp['market']
    comp_bal['Playoffs'] = np.where(temp['made_playoffs'] == 0, 'N','Y')
    
    return comp_bal

sbc_comp_bal = comp_balance('SBC')
st.table(sbc_comp_bal)

In [41]:
sbc_comp_bal

,Team,Total Revenue,Media Revenue,Media Revenue %,Other Revenue %,Market Size,Playoffs
team_id,,,,,,,
31,Carolina Swamp Foxes,165521010,75.0,45.3,54.7,4,N
32,Atlanta Braves,149058836,75.0,50.3,49.7,5,Y
33,Baltimore Orioles,143393177,87.5,61.0,39.0,7,N
34,Boston Red Sox,172098109,102.5,59.6,40.4,9,N
35,Nashville Sounds,209130282,102.5,49.0,51.0,8,N
36,Chicago Cubs,211100001,112.5,53.3,46.7,9,N
37,Cincinnati Reds,153271495,80.0,52.2,47.8,5,Y
38,Cleveland Spiders,245257304,97.5,39.8,60.2,7,Y
39,Vancouver Rockies,196932113,97.5,49.5,50.5,7,N


In [167]:
def teams(league):
    teams = pd.read_csv(leagues[league][3] + 'teams.csv')[['name', 'nickname','team_id','sub_league_id', 'division_id']]
    teams = teams[teams['team_id'] < 80]
    teams['league'] = np.where(teams['sub_league_id'] == 0, "AL", "NL")
    teams['division'] = np.select(
    [
        (teams['sub_league_id'] == 0) & (teams['sub_league_id'] == 0),
        (teams['sub_league_id'] == 0) & (teams['sub_league_id'] == 1),
        (teams['sub_league_id'] == 0) & (teams['sub_league_id'] == 2),
        (teams['sub_league_id'] == 1) & (teams['sub_league_id'] == 0),
        (teams['sub_league_id'] == 1) & (teams['sub_league_id'] == 1),
        (teams['sub_league_id'] == 1) & (teams['sub_league_id'] == 2)
    ],
    [
        'East',
        'Central',
        'West',
        'East',
        'Central',
        'West'
    ],
    default = 'Unknown'
    )

    teams['team'] = teams['name'] + ' ' + teams['nickname']

    return teams[['team', 'league', 'division']].set_index(teams['team_id'])

In [340]:
home_games = pd.read_csv(leagues[league][3] + 'games.csv')
home_games['date'] =  pd.to_datetime(home_games['date'], format='%Y/%m/%d')
home_games = home_games[(home_games['league_id'] == leagues[league][0]) & 
                        (home_games['game_type'] == 0) &
                        (home_games['date'].between(leagues[league][1], curr_date, inclusive = True))]
home_games['home_id'] = home_games['home_team']
home_games['home_runs'] = home_games['runs1']

home_games = home_games[['home_id','date', 'home_runs', 'innings']].set_index(home_games['game_id'])


away_games = pd.read_csv(leagues[league][3] + 'games.csv')
away_games['date'] =  pd.to_datetime(away_games['date'], format='%Y/%m/%d')
away_games = away_games[(away_games['league_id'] == leagues[league][0]) & 
                        (away_games['game_type'] == 0) &
                        (away_games['date'].between(leagues[league][1], curr_date, inclusive = True))]
away_games['away_id'] = away_games['away_team']
away_games['away_runs'] = away_games['runs0']

away_games = away_games[['away_id','date', 'away_runs', 'innings']].set_index(away_games['game_id'])

games_columns = away_games.columns.difference(home_games.columns)

games = home_games.merge(away_games[games_columns], on = 'game_id').merge(
    teams(league), how = 'left', left_on = 'home_id', right_on = 'team_id').rename(
    columns={'team':'home_team', 'league':'home_league', 'division':'home_division'}).merge(
    teams(league), how = 'left', left_on = 'away_id', right_on = 'team_id').rename(
    columns={'team':'away_team', 'league':'away_league', 'division':'away_division'})

all_games = games.copy()
all_games['hw'] = np.where(all_games['home_runs'] > all_games['away_runs'],1,0)
all_games['hl'] = np.where(all_games['home_runs'] < all_games['away_runs'],1,0)
all_games['aw'] = np.where(all_games['home_runs'] < all_games['away_runs'],1,0)
all_games['al'] = np.where(all_games['home_runs'] > all_games['away_runs'],1,0)

all_games['hd'] = all_games.home_runs - all_games.away_runs
all_games['ad'] = all_games.away_runs - all_games.home_runs

all_games['hdw'] = np.where(
    (all_games['home_division'] == all_games['away_division']) &
    (all_games['home_league'] == all_games['away_league']) &
    (all_games['home_runs'] > all_games['away_runs']),1,0)

all_games['hdl'] = np.where(
    (all_games['home_division'] == all_games['away_division']) &
    (all_games['home_league'] == all_games['away_league']) &
    (all_games['home_runs'] < all_games['away_runs']),1,0)

all_games['adw'] = np.where(
    (all_games['home_division'] == all_games['away_division']) &
    (all_games['home_league'] == all_games['away_league']) &
    (all_games['home_runs'] < all_games['away_runs']),1,0)

all_games['adl'] = np.where(
    (all_games['home_division'] == all_games['away_division']) &
    (all_games['home_league'] == all_games['away_league']) &
    (all_games['home_runs'] > all_games['away_runs']),1,0)

all_games['hcw'] = np.where(
    (all_games['home_league'] == all_games['away_league']) &
    (all_games['home_runs'] > all_games['away_runs']),1,0)

all_games['hcl'] = np.where(
    (all_games['home_league'] == all_games['away_league']) &
    (all_games['home_runs'] < all_games['away_runs']),1,0)

all_games['acw'] = np.where(
    (all_games['home_league'] == all_games['away_league']) &
    (all_games['home_runs'] < all_games['away_runs']),1,0)

all_games['acl'] = np.where(
    (all_games['home_league'] == all_games['away_league']) &
    (all_games['home_runs'] > all_games['away_runs']),1,0)

all_games['hiw'] = np.where(
    (all_games['home_league'] != all_games['away_league']) &
    (all_games['home_runs'] > all_games['away_runs']),1,0)

all_games['hil'] = np.where(
    (all_games['home_league'] != all_games['away_league']) &
    (all_games['home_runs'] < all_games['away_runs']),1,0)

all_games['aiw'] = np.where(
    (all_games['home_league'] != all_games['away_league']) &
    (all_games['home_runs'] < all_games['away_runs']),1,0)

all_games['ail'] = np.where(
    (all_games['home_league'] != all_games['away_league']) &
    (all_games['home_runs'] > all_games['away_runs']),1,0)

all_games['hexw'] = np.where((all_games.innings > 9) & (all_games['home_runs'] > all_games['away_runs']),1,0)
all_games['hexl'] = np.where((all_games.innings > 9) & (all_games['home_runs'] < all_games['away_runs']),1,0)
all_games['aexw'] = np.where((all_games.innings > 9) & (all_games['home_runs'] < all_games['away_runs']),1,0)
all_games['aexl'] = np.where((all_games.innings > 9) & (all_games['home_runs'] > all_games['away_runs']),1,0)

all_games['h1rw'] = np.where(all_games.home_runs - all_games.away_runs == 1,1,0)
all_games['h1rl'] = np.where(all_games.away_runs - all_games.home_runs == 1,1,0)
all_games['a1rw'] = np.where(all_games.away_runs - all_games.home_runs == 1,1,0)
all_games['a1rl'] = np.where(all_games.home_runs - all_games.away_runs == 1,1,0)

all_games['h2rw'] = np.where(all_games.home_runs - all_games.away_runs == 2,1,0)
all_games['h2rl'] = np.where(all_games.away_runs - all_games.home_runs == 2,1,0)
all_games['a2rw'] = np.where(all_games.away_runs - all_games.home_runs == 2,1,0)
all_games['a2rl'] = np.where(all_games.home_runs - all_games.away_runs == 2,1,0)

all_games['h3rw'] = np.where(all_games.home_runs - all_games.away_runs == 3,1,0)
all_games['h3rl'] = np.where(all_games.away_runs - all_games.home_runs == 3,1,0)
all_games['a3rw'] = np.where(all_games.away_runs - all_games.home_runs == 3,1,0)
all_games['a3rl'] = np.where(all_games.home_runs - all_games.away_runs == 3,1,0)

all_games['h4prw'] = np.where(all_games.home_runs - all_games.away_runs > 3 ,1,0)
all_games['h4prl'] = np.where(all_games.away_runs - all_games.home_runs > 3,1,0)
all_games['a4prw'] = np.where(all_games.away_runs - all_games.home_runs > 3,1,0)
all_games['a4prl'] = np.where(all_games.home_runs - all_games.away_runs > 3,1,0)

In [341]:
home_results = all_games.copy()
home_game_cols = [col for col in home_results if col.startswith('h')]
home_results = home_results[home_game_cols]

away_results = all_games.copy()
away_game_cols = [cols for cols in away_results if cols.startswith('a')]
away_results = away_results[away_game_cols]

In [342]:
home_beta = home_results.groupby(['home_team']).sum()
home_beta['hw%'] = round((home_beta['hw'] / (home_beta['hw'] + home_beta['hl'])*100),1)
home_beta = home_beta.drop(columns=['home_runs','home_id'])
#home_beta.rename(columns={'home_team':'team'}, inplace=True)

away_beta = away_results.groupby(['away_team']).sum()
away_beta['aw%'] = round((away_beta['aw'] / (away_beta['aw'] + away_beta['al'])*100),1)
away_beta = away_beta.drop(columns=['away_runs','away_id'])
#away_beta.rename(columns={'away_team':'team'}, inplace=True)

In [343]:
home_beta.columns = home_beta.columns.str[1:]
away_beta.columns = away_beta.columns.str[1:]

In [344]:
results = home_beta.add(away_beta, axis ='columns')
results['w%'] = results['w'] / (results['w'] + results['l'])

In [345]:
results['Record'] = str(results['w']) + ' - ' + str(results['l'])

In [346]:
results

,w,l,d,dw,dl,cw,cl,iw,il,exw,...,1rw,1rl,2rw,2rl,3rw,3rl,4prw,4prl,w%,Record
home_team,,,,,,,,,,,,,,,,,,,,,
Arizona Athletics,33,19,62,29,17,29,17,4,2,4,...,8,6,3,4,3,3,19,6,0.634615,home_team\nArizona Athletics 33\nAtlant...
Atlanta Braves,30,23,8,26,22,26,22,4,1,2,...,4,5,7,3,7,6,12,9,0.566038,home_team\nArizona Athletics 33\nAtlant...
Baltimore Orioles,27,27,-20,26,26,26,26,1,1,4,...,12,8,4,6,4,3,7,10,0.500000,home_team\nArizona Athletics 33\nAtlant...
Boston Red Sox,19,34,-36,18,30,18,30,1,4,1,...,3,6,5,10,2,8,9,10,0.358491,home_team\nArizona Athletics 33\nAtlant...
Carolina Swamp Foxes,30,21,41,25,19,25,19,5,2,5,...,9,8,6,3,3,4,12,6,0.588235,home_team\nArizona Athletics 33\nAtlant...
Chicago Cubs,17,37,-67,15,36,15,36,2,1,0,...,4,9,6,11,2,6,5,11,0.314815,home_team\nArizona Athletics 33\nAtlant...
Cincinnati Reds,22,32,-49,20,27,20,27,2,5,5,...,7,10,6,3,4,5,5,14,0.407407,home_team\nArizona Athletics 33\nAtlant...
Cleveland Spiders,29,24,51,24,20,24,20,5,4,1,...,5,6,4,8,7,3,13,7,0.547170,home_team\nArizona Athletics 33\nAtlant...
Detroit Tigers,32,19,98,29,16,29,16,3,3,3,...,5,7,5,5,8,5,14,2,0.627451,home_team\nArizona Athletics 33\nAtlant...


In [334]:
results.to_excel('SBC Standings/full standings.xlsx')
home_beta.to_excel('SBC Standings/home standings.xlsx')
away_beta.to_excel('SBC Standings/away standings.xlsx')

In [ ]:
csv.writer()